<a href="https://colab.research.google.com/github/renatabmagro/ExploratoryAnalysis_Correlation/blob/main/ExploratoryAnalysis_Correlation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Script para Análise Exploratória das Variáveis Meteorológicas & Fenologia & Produtividade | Etapa I**

*Desenvolvido em: Google Colab + Rstudio*

Nesse script estão descritos os procedimentos de leitura, organização e análise dos dados realizados para a construção do artigo de análise exploratória. 


> A análise exploratória deste script consiste em algumas etapas principais: 

>> Leitura do bando de dados meteorológicos.

>> Teste de normalidade -> escolha do tipo de correlação.

>> Seleção e organização dos dados meteorológicos (seleção das variáveis, agrupamento das variáveis).

>> Execução das análises de correlação entre os dados meteorológicos e a produtividade do pomar de macieira; e entre os dados meteorológicos e a fenologia floral da macieira. 
 
------------------------------------------
> As análises foram realizadas em função da configuração dos dados meteorológicos: 
>> Dados meteorológicos mensais: 

  - Correlação Início da floração

  - Correlação Plena floração

  - Correlação Produtividade 

>> Dados meteorológicos sazonais: 

  - Correlação Início da floração 

  - Correlação Plena floração

  - Correlação Produtividade 




In [ ]:
# Instalação das bibliotecas 

install.packages("readxl")
library(readxl)
install.packages("tidyr")
install.packages("tidyverse")
library(tidyr)
library(tidyverse)
install.packages("dplyr")
library(dplyr)
install.packages("corrplot")
library(corrplot)
install.packages("ggcorrplot")
library(ggcorrplot)
install.packages("lubridate")
library(lubridate)
install.packages("ggplot2")
library(ggplot2)
install.packages("dygraphs")
library(dygraphs)
install.packages("cowplot")
library(cowplot)
install.packages("lares")
library(lares)
install.packages("GGally")
library(GGally)
install.packages("ggcorrplot")
library(ggcorrplot)

In [ ]:
#####################################################################################################

In [ ]:
# Leitura da planilha de dados - para execução do teste de normalidade 
dados_clima = read_excel("BancoDados_clima.xlsx") #dados climáticos em frequência diária (2008-2022)

In [ ]:
#visualizar a head dos dados climáticos
head(dados_clima)

In [ ]:
#visualizar período início-final dos dados
range(dados_clima$Data)

In [ ]:
#remoção coluna de Dia, Mês e Ano
dados_clima_ <- select(dados_clima, -Dia, -Mês, -Ano)
head(dados_clima_)

In [ ]:
#duplicando o data frame para conservar os dados originais
dados_clima2 <- dados_clima_
head(dados_clima2)

In [ ]:
#criar uma coluna de mês e ano no dados_clima2 -> função floor_data (pacote lubridate)
dados_clima2$year_month <- floor_date(dados_clima2$Data, 
                                      "month")

In [ ]:
#fazer acumulado das variáveis por mês (pacote dplyr)
data_aggr <- dados_clima2 %>%
  group_by(year_month) %>%
  dplyr::summarize(Temp_max = mean(Temp_max), Temp_min = mean(Temp_min), Temp_media = mean(Temp_media),
                    Radiacao_acum = mean(Radiacao_acum), Precip = sum(Precip), GD = sum(GD),
                    Eto = mean(Eto), HorasFrio = sum(HorasFrio)) %>%
                    as.data.frame()

In [ ]:
####################################################################
#    Resumo de como os dados foram agregados:
##   Tem_max, temp_min, temp_media, radiacao, eto -> média mensal
##   Precip, GD, HorasFrio -> soma mensal
####################################################################

In [ ]:
#remoção da coluna que representa a data (year_month) do dataframe, pois essa não é necessária para executar o teste de normalidade e a análise de correlação
data_corrigida <- select(data_aggr, -year_month)

In [ ]:
#verificar se existam NAs nos dados
is.na(data_corrigida) %>% table()

In [ ]:
#caso existam, fazer a remoção dos valores NA:
# data_corrigida <- na.omit(data_corr) #remoção valores NA

In [ ]:
# Teste de normalidade #
  ## Realizado o teste de Shapiro-Wilk para verificar se os dados seguem a distribuição normal. 
  ## Ho: os dados seguem a distribuição normal ; H1: os dados não seguem a distribuição normal. 

t1=shapiro.test(data_corrigida$Temp_max)
t2=shapiro.test(data_corrigida$Temp_min)
t3=shapiro.test(data_corrigida$Temp_media)
t4=shapiro.test(data_corrigida$Radiacao_acum)
t5=shapiro.test(data_corrigida$Precip)
t6=shapiro.test(data_corrigida$GD)
t7=shapiro.test(data_corrigida$Eto)
t8=shapiro.test(data_corrigida$HorasFrio)


variavel <- c(data_corrigida$Temp_max, data_corrigida$Temp_min, data_corrigida$Temp_media, data_corrigida$Radiacao_acum, data_corrigida$Precip,
              data_corrigida$GD, data_corrigida$Eto, data_corrigida$HorasFrio)

valorp <- c(t1$p.value, t2$p.value, t3$p.value, t4$p.value, t5$p.value, t6$p.value, t7$p.value, t8$p.value)

resultados <- cbind(valorp)
rownames(resultados) <- cbind("Temp_max", "Temp_min", "Temp_media", "Radiacao_acum", "Precip", "GD", "Eto", "Horas Frio")
print(resultados, digits = 4)



In [ ]:
######################################## 
##   Resultado teste de normalidade:
##   Rejeita-se Ho | Os dados não seguem a distribuição normal 
##   Devido à isso, deve-se utilizar o teste de correlação de Spearman
#########################################

In [ ]:
######################################################################################################

In [ ]:
## Inicio da Análise Exploratória ~~ DADOS METEOROLÓGICOS MENSAIS ##

  # - Correlação dados meteorológicos mensais & Produtividade (kg/ha)
    # -- Para executar essa correlação foram utilizados os dados meteorológicos de 2011-2020;
    # -- A produtividade avaliada corresponde a safra atual (n) e a safra seguinte (n+1)

# Leitura dos dados meteorológicos
BD_clima_EN <- read_excel('BD_clima_2011_2020_EN.xlsx')
# Head dos dados meteorológicos
head(BD_clima_EN)


In [ ]:
# Leitura dos dados de produtividade
BD_prod_EN <- read_excel('BD_producao_cultivar_EN.xlsx')
# Head dos dados de produtividade
head(BD_prod_EN)

In [ ]:
# Organizar os dados meteorológicos diários em acumulados de dados mensais 

  #criar coluna mês/ano
BD_clima_EN$year_month <- floor_date(BD_clima_EN$Date, 
                                  "month")
#criar uma nova planilha com os dados agregados
dados_climaticos_mensais <- BD_clima_EN %>%
  group_by(year_month) %>%
  dplyr::summarize(Tmax = mean(Tmax), Tmin = mean(Tmin), Tmean = mean(Tmean),
                    SRad = mean(SRad), Precip = sum(Precip), GDD = sum(GDD),
                    ET = mean(ET), Chill_hours = sum(Chill_hours)) %>%
                    as.data.frame()

In [ ]:
# unir dataframe dados meteorológicos mensais + dataframe produtividade
dados_clima_prod <- cbind(dados_climaticos_mensais, BD_prod_EN)
#dados_clima_prod

#selação das variáveis que efetivamente serão utilizadas (~exclusão das variáveis que não possuem utilidade)
dados_clima_prod <- select(dados_clima_prod, -year_month, -Date)

In [ ]:
#visualizar planilha
dados_clima_prod

In [ ]:
#verificar NAs
is.na(dados_clima_prod)%>% table
#dados_bind <-na.omit(dados_bind)

In [ ]:
#executar Correlação
  #pacote 'cor' e ggcorrplot; método 'spearman'

prod_clima_correlacao = cor(dados_clima_prod, method="spearman")

#plot grafico de correlação 
prod_clima_correlacao_grafico <- ggcorrplot(
                                            prod_clima_correlacao, 
                                            hc.order= TRUE,
                                            type = "lower",
                                            lab=TRUE,
                                            )

#visualizar resultado gráfico 
prod_clima_correlacao_grafico

In [ ]:
# Correlações rankeadas conforme o nível de significância 
  # Negative correlations are represented in red and positive correlations in blue.
  # pacote 'lares'

corr_cross(dados_clima_prod, # name of dataset
  method = "spearman",
  max_pvalue = 0.05, # display only significant correlations (at 5% level)
  top = 30 # display top 30 couples of variables (by correlation coefficient)
)

In [ ]:
###############################################################################

In [ ]:
## Continuação da Análise Exploratória ~~ DADOS METEOROLÓGICOS MENSAIS ##

  # - Correlação dados meteorológicos mensais & Início da floração 
    # -- Para executar essa correlação foram utilizados os dados meteorológicos de 2011-2020;
    # -- O início da floração foi avaliada com relação à safra atual (n) e a safra seguinte (n+1)

# Leitura dos dados meteorológicos
  #BD_clima_EN <- read_excel('BD_clima_2011_2020_EN.xlsx')
# Head dos dados meteorológicos
 #head(BD_clima_EN)


In [ ]:
# Leitura dos dados de início da floração 
  # As datas de início da floração estão apresentadas em dias julianos 
dados_inicio_flor <- read_excel("BD_fenologia_cultivar2_ENG.xlsx")

# Head dos dados de início da floração 
head(dados_inicio_flor)

In [ ]:
# unir dataframe dos dados climaticos mensais + dataframe dos dados de inicio da floração 
dados_clima_flor <- cbind(dados_climaticos_mensais, dados_inicio_flor)

#selação das variáveis que efetivamente serão utilizadas (~exclusão das variáveis que não serão)
dados_clima_flor <- select(dados_clima_flor, -year_month, -Date, -FB_GALA, -Next_FB_GALA, -FB_FUJI, -Next_FB_FUJI)
#dados_clima_flor

In [ ]:
#verificar NAs
is.na(dados_clima_flor)%>% table
#dados_bind <-na.omit(dados_bind)

In [ ]:
# Executar Correlação entre dados meteorológicos mensais e as datas de início de floração (para a cultivar Fuji e Gala)
  # Pacote 'ggcorrplot'
flor_clima_correlacao = cor(dados_clima_flor, method="spearman")

#plot grafico
flor_clima_correlacao_grafico <- ggcorrplot(
                                            flor_clima_correlacao, 
                                            hc.order= TRUE,
                                            type = "lower",
                                            lab=TRUE,
                                             )

# visualizar gráfico 
flor_clima_correlacao_grafico

In [ ]:
# Verificar correlações rankeadas; considerando P<0.05
corr_cross(dados_clima_flor, # name of dataset
  method="spearman",
  max_pvalue = 0.05, # display only significant correlations (at 5% level)
  top = 60 # display top 60 couples of variables (by correlation coefficient)
)

In [ ]:
###############################################################################

In [ ]:
## Continuação da Análise Exploratória ~~ DADOS METEOROLÓGICOS MENSAIS ##

  # - Correlação dados meteorológicos mensais & Plena floração 
    # -- Para executar essa correlação foram utilizados os dados meteorológicos de 2011-2020;
    # -- A Plena floração foi avaliada com relação à safra atual (n) e a safra seguinte (n+1)

# Leitura dos dados meteorológicos
  #BD_clima_EN <- read_excel('BD_clima_2011_2020_EN.xlsx')
# Head dos dados meteorológicos
 #head(BD_clima_EN)

In [ ]:
# Leitura dos dados de Plena floração 
  # As datas de Plena floração estão apresentadas em dias julianos 
dados_plena_flor <- read_excel("BD_fenologia_cultivar2_ENG.xlsx")

# Head dos dados de início da floração 
head(dados_plena_flor)

In [ ]:
# unir dataframe dados meteorológicos mensais + dataframe fenologia -> plena floração 
dados_clima_PF <- cbind(dados_climaticos_mensais, dados_plena_flor)
#dados_clima_prod

#selação das variáveis que efetivamente serão utilizadas; apenas relacionadas à plena floração (~exclusão das variáveis que não serão utilizadas)
dados_clima_PF <- select(dados_clima_PF, -year_month, -Date, -Flower_GALA, -Next_Flower_GALA, -Flower_FUJI, -Next_Flower_FUJI)

#visualizar o df
head(dados_clima_PF)

In [ ]:
#verificar NAs
is.na(dados_clima_PF)%>% table
#dados_bind <-na.omit(dados_bind)

In [ ]:
# Executar Correlação entre dados meteorológicos mensais e datas de plena floração

PF_correlacao = cor(dados_clima_PF, method="spearman")

#plot grafico
PF_correlacao_grafico <- ggcorrplot(
                                     PF_correlacao, 
                                     hc.order= TRUE,
                                     type = "lower",
                                     lab=TRUE,
                                     )

# Visualizar o gráfico 
PF_correlacao_grafico

In [ ]:
# Verificar correlações rankeadas; p<0.05

corr_cross(dados_clima_PF, # name of dataset
  method="spearman",
  max_pvalue = 0.05, # display only significant correlations (at 5% level)
  top = 60 # display top 60 couples of variables (by correlation coefficient)
)

#apenas 23 correlações foram identificadas

In [ ]:
################################################################################

In [ ]:
## Continuação da Análise Exploratória ~~ DADOS METEOROLÓGICOS SAZONAIS ##
    # a partir de agora, as análises de correlação foram realizadas considerando os dados meteorológicos agregados por estações climáticas 

  # - Correlação dados meteorológicos sazonais & Produtividade (kg/ha) 
    # -- Para executar essa correlação foram utilizados os dados meteorológicos de 2011-2020 acumulados em função das estações climáticas;
    # -- A Produtividade foi avaliada com relação à safra atual (n) e a safra seguinte (n+1)

# Leitura dos dados meteorológicos sazonais & dados de produtividade (variável dependente)
  BD_season_prod_EN <- read_excel('BD_estacoesclima_producao_ENG.xlsx')
# Head dos dados meteorológicos
 head(BD_season_prod_EN)

In [ ]:
# Ajuste da planilha:
  # Retirada da coluna 'year' (não é necessária para as correlações) e retirada da coluna 'Chill_Hours_summer' (só contém zero, então não é necessária)

# Duplicar arquivo e retirar as variáveis que não serão utilizadas
dados_season_2 <- select(BD_season_prod_EN, -Year, -Chill_Hours_summer)


In [ ]:
#execução da correlação

correlacao_season = cor(dados_season_2, method = "spearman")

#plot do gráfico 

ggcorrplot(
  correlacao_season, 
  hc.order= TRUE,
 type = "lower",
  #lab=TRUE,
) +
ggplot2::theme(
          axis.text.x = element_text(angle = 90))

In [ ]:
#visualizar tabela de correlação
correlacao_season

In [ ]:
#Gráfico para apresentar as variáveis que tem maior correlação 
  #corr_var() function to focus on the correlation of one variable against all others, and return the highest ones in a plot:

plotGALA <- corr_var(dados_season_2, # name of dataset
  Yield_GALA, # name of variable to focus on
  method = "spearman",
  top = 10 # display top 10 correlations
)

plot_prox_GALA <- corr_var(dados_season_2, # name of dataset
  NextYield_GALA, # name of variable to focus on
  method = "spearman",
  top = 10 # display top 10 correlations
)

plotFUJI <- corr_var(dados_season_2, # name of dataset
            Yield_FUJI, # name of variable to focus on
           method = "spearman",
           top = 10 # display top 10 correlations
           )

plot_prox_FUJI <- corr_var(dados_season_2, # name of dataset
  NextYield_FUJI, # name of variable to focus on
  method = "spearman",
  top = 10 # display top 10 correlations
)

In [ ]:
# Visualizar gráficos Produtividade - Gala
plotGALA
plot_prox_GALA

In [ ]:
# Visualizar gráficos Produtividade - Fuji
plotFUJI
plot_prox_FUJI

In [ ]:
###############################################################################

In [ ]:
## Continuação da Análise Exploratória ~~ DADOS METEOROLÓGICOS SAZONAIS ##
    
  # - Correlação dados meteorológicos sazonais & Início da Floração 
    # -- Para executar essa correlação foram utilizados os dados meteorológicos de 2011-2020;
    # -- O Início da floração foi avaliada com relação à safra atual (n) e a safra seguinte (n+1)

# Leitura dos dados meteorológicos
BD_season_flor_EN <- read_excel('BD_estacoesclima_fenologia2_EN.xlsx')
# Head dos dados meteorológicos
head(BD_season_flor_EN)

In [ ]:
#remover variáveis que não serão utilizadas
dados_season_fenologica_2 <- select(BD_season_flor_EN, -Chill_Hours_summer, -Year)

In [ ]:
#execução da correlação

correlacao_season_fen = cor(dados_season_fenologica_2, method = "spearman")

#plot do gráfico 

ggcorrplot(
  correlacao_season_fen, 
  hc.order= TRUE,
 type = "lower",
  #lab=TRUE,
) +
ggplot2::theme(
          axis.text.x = element_text(angle = 90))

In [ ]:
##############################################################################################
# Etapa de visualização dos resultados da correlação entre Dados sazonais & Início de floração

In [ ]:
corr_cross(dados_season_fenologica_2, # name of dataset
  method = "spearman",
  max_pvalue = 0.05, # display only significant correlations (at 5% level)
  top = 40 # display top 10 couples of variables (by correlation coefficient)
)

In [ ]:
#Gráfico para apresentar as variáveis que tem maior correlação 
  #corr_var() function to focus on the correlation of one variable against all others, and return the highest ones in a plot:

plotFlorGALA <- corr_var(dados_season_fenologica_2, # name of dataset
  Flower_GALA, # name of variable to focus on
  method = "spearman",
  top = 5 # display top 5 correlations
)

plot_proxFlor_GALA <- corr_var(dados_season_fenologica_2, # name of dataset
  Next_Flower_GALA, # name of variable to focus on
  method = "spearman",
  top = 5 # display top 5 correlations
)

plotFlorFUJI <- corr_var(dados_season_fenologica_2, # name of dataset
            Flower_FUJI, # name of variable to focus on
           method = "spearman",
           top = 5 # display top 5 correlations
           )

plot_proxFlor_FUJI <- corr_var(dados_season_fenologica_2, # name of dataset
  Next_Flower_FUJI, # name of variable to focus on
  method = "spearman",
  top = 5 # display top 5 correlations
)

In [ ]:
# Visualizar gráficos Início da floração - Gala
plotFlorGALA
plot_proxFlor_GALA

In [ ]:
# Visualizar gráficos Início da floração - Fuji
plotFlorFUJI
plot_proxFlor_FUJI 

In [ ]:
################################################################################

In [ ]:
## Continuação da Análise Exploratória ~~ DADOS METEOROLÓGICOS SAZONAIS ##
    # a partir de agora, as análises de correlação foram realizadas considerando os dados meteorológicos agregados por estações climáticas 

  # - Correlação dados meteorológicos sazonais & Plena Floração
    # -- Para executar essa correlação foram utilizados os dados meteorológicos de 2011-2020;
    # -- A Plena floração foi avaliada com relação à safra atual (n) e a safra seguinte (n+1)

# Leitura dos dados meteorológicos sazonais e de Plena floração
dados_season_plenaFlor <- read_excel("BD_estacoesclima_fenologia_plenaflor_EN.xlsx")
# Head dos dados meteorológicos
head(dados_season_plenaFlor)

In [ ]:
df_season_plenaFlor <- select(dados_season_plenaFlor, -Chill_Hours_summer, -Year)

In [ ]:
#execução da correlação

correlacao_season_PF = cor(df_season_plenaFlor, method = "spearman")

#plot do gráfico 

ggcorrplot(
  correlacao_season_PF, 
  hc.order= TRUE,
 type = "lower",
  #lab=TRUE,
) +
ggplot2::theme(
          axis.text.x = element_text(angle = 90))

In [ ]:
##########################################################################################
# Etapa de visualização dos resultados da correlação entre dados sazonais & Plena floração

In [ ]:
#Function to compute all correlations and return the highest and significant ones in a plot:
#Negative correlations are represented in red and positive correlations in blue.

corr_cross(df_season_plenaFlor, # name of dataset
  method ="spearman",
  max_pvalue = 0.05, # display only significant correlations (at 5% level)
  top = 50 # display top 10 couples of variables (by correlation coefficient)
)

In [ ]:
#Gráfico para apresentar as variáveis que tem maior correlação 
  #corr_var() function to focus on the correlation of one variable against all others, and return the highest ones in a plot:

plotGALA_FB <- corr_var(df_season_plenaFlor, # name of dataset
  FB_GALA, # name of variable to focus on
  method = "spearman",
  top = 5 # display top 5 correlations
)

plot_prox_GALA_FB <- corr_var(df_season_plenaFlor, # name of dataset
  Next_FB_GALA, # name of variable to focus on
  method = "spearman",
  top = 5 # display top 5 correlations
)

plotFUJI_FB <- corr_var(df_season_plenaFlor, # name of dataset
            FB_FUJI, # name of variable to focus on
           method = "spearman",
           top = 5 # display top 5 correlations
           )

plot_prox_FUJI_FB <- corr_var(df_season_plenaFlor, # name of dataset
  Next_FB_FUJI, # name of variable to focus on
  method = "spearman",
  top = 5 # display top 5 correlations
)

In [ ]:
#Visualizar gráficos Plena Floração - Gala
plotGALA_FB 
plot_prox_GALA_FB

In [ ]:
#Visualizar gráficos Plena Floração - Fuji
plotFUJI_FB
plot_prox_FUJI_FB